# Demo of _narrative graphs_


In [1]:
!rm output/demo_db.sqlite

In [2]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("rmisra/news-category-dataset", "News_Category_Dataset_v3.json")
data = pd.read_json(path, lines=True)

/Users/au479461/PycharmProjects/narrative-graph/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [5]:
sample = data.sample(2500)
sample["text"] = sample["headline"] + "\n" + sample["short_description"]
docs = sample["text"].tolist()
categories = sample["category"].tolist()
timestamps = sample["date"].tolist()

In [1]:
from narrativegraph import NarrativeGraph

model = NarrativeGraph(sqlite_db_path="output/demo_db.sqlite")
model.fit(docs, categories=categories, timestamps=timestamps)

In [10]:
server = model.serve_visualizer(block=False)

INFO:root:Server started in background on port 8001
INFO:     Started server process [62909]
INFO:     Waiting for application startup.
INFO:root:Database service provided to state before startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:58776 - "GET /vis HTTP/1.1" 307 Temporary Redirect


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-20' coro=<BackgroundServer._run_server() done, defined at /Users/au479461/PycharmProjects/narrative-graph/narrativegraph/server/backgroundserver.py:22> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/Users/au479461/PycharmProjects/narrative-graph/narrativegraph/server/backgroundserver.py", line 38, in start
    self._server = None
  File "/Users/au479461/PycharmProjects/narrative-graph/.venv/lib/python3.12/site-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/au479461/PycharmProjects/narrative-graph/.venv/lib/python3.12/site-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/Users/au479461/PycharmProjects/narrative-graph/.venv/lib/python3.12/site-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/Library/Frameworks/Python.fram

INFO:     127.0.0.1:58776 - "GET /graph/bounds HTTP/1.1" 200 OK
INFO:     127.0.0.1:58778 - "POST /graph HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:58778 - "POST /graph/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:58786 - "POST /graph HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:58786 - "POST /graph/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [62909]


In [11]:
server.show_iframe()

In [12]:
server.stop()

INFO:root:Background server stopped
